In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re
import string

url = "https://raw.githubusercontent.com/sahalmaghfud/MLLawanJudi/main/datasetB.csv"
df = pd.read_csv(url)

texts = df['text'].values
labels = df['label'].values


In [4]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.1,shuffle=True)

print(f" training: {len(X_train)}")
print(f"testing: {len(X_test)}")

print(X_test)



 training: 180
testing: 21
['slot gacor online  mainkan berbagai permainan terbaik'
 'free spin dan buy spin tanpa limit' 'mainkan di mana saja'
 'manisnya cinta dalam setiap gigitan' 'mobilmobilan paling keren'
 'lagi gacor boss  gate of olympus pestä gratlsan'
 'semua hiburan favoritmu' 'daftar sekarang joker game slot joker 123'
 'bergabunglah di kota88 dan nikmati bonus member baru slot 150%!'
 'slot online gacor resmi indonesia proses cepat 2 menit pendaftaran mudah tingkat kemenangan layanan onlne 24 jam min. deposit 25.000'
 'atasi sakit kepala dengan cepat' 'mengirim dengan hati'
 'jaya slot4d slot gacor gampang maxwin promo bonus new member 25k'
 'live in your world play in ours' 'kayak ada lemon limenya gitu no bokis'
 'indomaret dekat di hati mudah dijangkau'
 'selamat datang di 2 3 4 betting tariuhan online terlengkap & terpercaya daftar gratis'
 'jandaslot88 ayo daftar sekarang juga!'
 'menang berapapun pasti di bayar langsung'
 'sendok88 akses situs game online resmi & tr

In [6]:
# Constants
MAX_LENGTH = 20
MAX_VOCAB_SIZE = 1000
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding=padding_type, truncating=trunc_type)

X_test_seq = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding=padding_type, truncating=trunc_type)

vocab_size = len(tokenizer.word_index) + 1

print(f"Vocabulary size: {vocab_size}")


Vocabulary size: 555


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(MAX_LENGTH,)),
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=100),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 100)             │          55,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 16, 64)              │          32,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d                 │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 88,621 (346.18 KB)

 Trainable params: 88,621 (346.18 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
print("X_train_pad shape:", X_train_pad.shape)
print("y_train shape:", y_train.shape)
print("X_test_pad shape:", X_test_pad.shape)
print("y_test shape:", y_test.shape)

X_train_pad shape: (180, 20)
y_train shape: (180,)
X_test_pad shape: (21, 20)
y_test shape: (21,)


In [9]:
history = model.fit(X_train_pad, y_train, epochs=30, validation_data=(X_test_pad, y_test))

Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.3910 - loss: 0.6968 - val_accuracy: 0.7143 - val_loss: 0.6913
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7657 - loss: 0.6838 - val_accuracy: 0.6667 - val_loss: 0.6874
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7983 - loss: 0.6757 - val_accuracy: 0.6667 - val_loss: 0.6811
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8542 - loss: 0.6668 - val_accuracy: 0.7143 - val_loss: 0.6758
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8791 - loss: 0.6531 - val_accuracy: 0.7143 - val_loss: 0.6708
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8572 - loss: 0.6365 - val_accuracy: 0.7619 - val_loss: 0.6638
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9223 - loss: 0.6234 - val_accuracy: 0.7619 - val_loss: 0.6538
Epoch 8/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9137 - loss: 0.5982 - val_accuracy: 0.8571 - val_loss: 0.6433
Epoch 

In [22]:
new_text = ["Hidupkan Semangatmu dengan coke ayo"]
new_text_seq = tokenizer.texts_to_sequences(new_text)
new_text_pad = pad_sequences(new_text_seq, maxlen=MAX_LENGTH)

prediction = model.predict(new_text_pad)
print(prediction)
print(f"Prediksi: {'Judi Online' if prediction > 0.5 else 'Bukan Judi Online'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
[[0.0163912]]
Prediksi: Bukan Judi Online


In [21]:
new_text = ["main di maisan777 gacor selalu"]
new_text_seq = tokenizer.texts_to_sequences(new_text)
new_text_pad = pad_sequences(new_text_seq, maxlen=MAX_LENGTH)

prediction = model.predict(new_text_pad)
print(prediction)
print(f"Prediksi: {'Judi Online' if prediction > 0.5 else 'Bukan Judi Online'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[[0.8840974]]
Prediksi: Judi Online


In [24]:
from google.colab import files
tf.keras.models.save_model(model, 'modelLajuB.keras')
files.download('modelLajuB.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>